In [2]:
import pandas as pd

# 7. Students Do: compete with your classmates to create a model (any algorithm(s) of your choice) with the highest R value using the boston dataset. You may use any supplemental data to increase performance but you MUST use the train:test:val split below:

In [10]:
import sklearn.ensemble

In [3]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.datasets import load_boston

X, y = load_boston(True)
X = pd.DataFrame(X, columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO',
 'B', 'LSTAT'])
len(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=310)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=.5, random_state=310)

model = RandomForestRegressor(n_estimators=400, oob_score=True, n_jobs=-1, random_state=310)
model.fit(X_train, y_train)
model.score(X_train, y_train), model.score(X_test, y_test)

model = 

(0.9836355121337571, 0.8912554568184679)

# 8. All about Random Forests and some of their related models

### Recall that random forests are build upon many individual trees that are each trained on a subset of the data (both in terms of features and datapoints)

### We can tune the parameters are each decision tree to affect the overall level of detail captured.

### Some of these hyperparameters are as follows:

 - n_estimators: default 'warn'. This is the number of trees in your model. The goal here is to pick the minimum amount of trees to achieve the lowest generalized loss. I usually pick ~500 to start and experiment with lower and higher numbers. the process of choosing trees is known as pruning. read more here: 
 - criterion: 'mse' or 'mae'. mse is differentiable while mae is not. mse also penalizes samples that are very far.
 - max_depth: refers to how many layers the tree can make. If you find your model is overfitting, limiting this can help limit overfitting.
 - min_samples_split: self explanatory,
 - min_samples_leaf: self explanatory, 
 - min_weight_fraction_leaf=0.0, 
 - max_features: default sqrt(n). How many features to consider for each tree at each split. This is super important to tune for each model as it can drastically improve (or decrease) the performance of your model. 
 - max_leaf_nodes=None, 
 - bootstrap: should i pick a subset of samples to train on? default True 
 - oob_score: out of bag score. will test trees on samples held out on training that tree,
 - n_jobs=1, 
 - random_state: self explanatory,
 - verbose: self explanatory, 
 - warm_start: add the ability to continue training the model after it has already been trained. , 
 - class_weight=None)
 
 Important notes:
 - Great in high dimensions
 - slow to build and predict
 - more black-box than linear regression algorithms
 - Lesson from Random Forests is: when in doubt, use an ensemble.
 - Good for sparse data (Linear regression is not).
 
 ## Gradient Boosted Trees are ensembles of traditionally small, weak trees that perform poorly individually, but well as an ensemble.
 
 0. We begin with a weak learner (AKA shallow tree)
 1. We represent the loss of a tree in a differentiable equation
 2. We take the gradient of previous equation to find the optimal parameters.
 3. We add it to the ensemble of trees.
 4. repeat.
 
 Important notes:
 - Single threaded
 - Slow to build, but fast to predict
 - can perform better than Random Forests.
 
 ## XGBoost
 
 Paper: https://arxiv.org/pdf/1603.02754.pdf?source=post_page---------------------------
 
 Similar to gradient boosted trees, but also implements
 - pruning
 - adds some paralellizability
 - adds some regularization through L1 and L2 loss
 - some more fancy algorithms such as Weighted Quantile Sketch (which I don't know much about, yet)

# >>>>> NO FREE LUNCH! <<<<<

In [473]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
#import xgboost

import numpy as np

In [609]:
file_to_load = "Dataframe519.csv"
df = pd.read_csv(file_to_load, sep="|")
df = df.drop(['Unnamed: 0'],axis=1)
# need valid features/ eliminate NAN features
df = df.loc[df['features'].isnull()==False, :]
Y = np.array(df['Price'])
X = df.drop(['Price'],axis=1)


In [497]:
len(Y)

56192

In [476]:
len(X)

56192

In [612]:
#Mapping 'features' to numeric values
mapFeatures = dict()
features = X.features.str.split(',',  expand=True)
features = features[features.columns].replace('[\(,\)]', '', regex=True)
features.to_csv('featuresHuman.csv')

 

weightValues = 0
# 
for value in range(0, features.shape[1], 1):
    features[value] = features[value].apply(str)
    #print(features[value])
    for each in features[value].unique():
        if not (each in mapFeatures.keys()):             
            mapFeatures[each] = weightValues             
            weightValues = weightValues +1
    
mapFeatures

{'Rear Spoiler': 0,
 'Cruise Control': 1,
 'Traction Control': 2,
 'Auxiliary Audio Input': 3,
 'AM/FM Stereo': 4,
 'Air Conditioning': 5,
 'Overhead Airbags': 6,
 'Special Edition': 7,
 'Skylights': 8,
 'Sunroofs': 9,
 'Alloy Wheels': 10,
 'Leather Seats': 11,
 'Front Seat Heaters': 12,
 'Convenience Package': 13,
 'MACH Audio': 14,
 'Satellite Radio Ready': 15,
 'Rockford Fosgate Sound': 16,
 'Pioneer Sound System': 17,
 'ABS Brakes': 18,
 'Luxury Package': 19,
 'Tow Hitch': 20,
 'Platinum Edition': 21,
 'Power Sliding Doors': 22,
 'Touring Package': 23,
 'Rear View Camera': 24,
 'Leatherette Seats': 25,
 'Running Boards': 26,
 'Premium Package': 27,
 'Sport Package': 28,
 'Soft Top': 29,
 '4WD/AWD': 30,
 'Navigation System': 31,
 'Panoramic Sunroof': 32,
 'BOSE Sound System': 33,
 'Turbo Charged Engine': 34,
 'Parking Sensors': 35,
 'Full Roof Rack': 36,
 'Fold-Away Third Row': 37,
 'Grand Touring Package': 38,
 'DVD Video System': 39,
 'Cold Weather Package': 40,
 'Alpine Sound Sys

In [613]:
#long computation values due to vast assignment
for key in mapFeatures:
    for value in range(0, features.shape[1], 1):
        features[value] = features[value].replace(key, mapFeatures[key], regex=True)
    


In [479]:
#columnsName = ['Feature'+ str(each) for each in list(features.columns)]
columnsName = ['Feature'+ str(each) for each in range(0, len(features.columns), 1)]
columnsName
features.columns = columnsName

for info in columnsName:
    features[info] = pd.to_numeric(features[info], errors='coerce')

features.dtypes
features.head(5)

,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5
0,0.0,6.0,78.0,92.0,179.0,182.0
1,1.0,3.0,0.0,5.0,98.0,180.0
2,2.0,78.0,18.0,98.0,4.0,92.0
3,3.0,6.0,78.0,4.0,92.0,157.0
4,4.0,92.0,179.0,182.0,184.0,173.0


In [615]:
features.to_csv('FeaturesML.csv')

In [480]:
len(features)

56192

In [481]:
# Quatify specs into numberic categories
#4 Cylinders, 2WD, 1.6L Engine, Manual 5 Speed ...

specs = X['specs'].str.split(' ',  expand=True)
specs.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11
0,4,"Cylinders,","2WD,",1.6L,"Engine,",Manual,5,Speed,"Transmission,",87K,"Miles,",Red/Black
1,4,"Cylinders,","2WD,",2.2L,"Engine,",Manual,5,Speed,"Transmission,",127K,"Miles,",Gray/Gray
2,3,"Cylinders,","2WD,",1.0L,"Engine,",Automatic,"Transmission,",86K,"Miles,",White/Black,None,None
3,4,"Cylinders,","2WD,",1.6L,"Engine,",Manual,5,Speed,"Transmission,",114K,"Miles,",Silver/Black
4,4,"Cylinders,","2WD,",1.6L,"Engine,",Manual,5,Speed,"Transmission,",73K,"Miles,",White/Black


In [482]:

#Assigment issues

correction = specs.loc[specs[0].str.contains('WD')==True, :] 
correction = (correction).shift(periods=2, axis='columns')
correction[[0]] = -1
correction[[1]] = 'Cylinders'
specs.loc[specs[0].str.contains('WD')==True, :] = correction


# Replace Nan with "" 
# Car Color
specs[11] = specs[11].fillna("")
specs.loc[specs[9].str.contains('K')==True, [9]] = ""
specs[11] = specs[9]+specs[11]

# Automatic = 0, Manual >= 5 speed upward
specs.loc[specs[6].str.contains('Transmission')==True, [6]] = 0
#specs.to_csv("specsDebug.csv")

specs.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11
0,4,"Cylinders,","2WD,",1.6L,"Engine,",Manual,5,Speed,"Transmission,",,"Miles,",Red/Black
1,4,"Cylinders,","2WD,",2.2L,"Engine,",Manual,5,Speed,"Transmission,",,"Miles,",Gray/Gray
2,3,"Cylinders,","2WD,",1.0L,"Engine,",Automatic,0,86K,"Miles,",White/Black,None,White/Black
3,4,"Cylinders,","2WD,",1.6L,"Engine,",Manual,5,Speed,"Transmission,",,"Miles,",Silver/Black
4,4,"Cylinders,","2WD,",1.6L,"Engine,",Manual,5,Speed,"Transmission,",,"Miles,",White/Black


In [483]:
X.loc[56008, :]['specs']

'2WD, 5.3L Engine, Automatic Transmission, 12K Miles, Black/Black'

In [484]:
specs.to_csv("specsDebug2.csv")
#specs.loc[311, :]

In [485]:
len(specs)
#len(features)

56192

In [486]:
#Drop names
specs.drop([1,4,5, 7,8,9,10], axis=1, inplace=True)


#0 > 'Cylinders'> 4 to 8 cylinders
#2 > 'DriveTrain' > 2= 2WD or 4= 4WD
#4 > 'Engine' > 1.6L, 2.2L etc.
#6 > 'Transmission' : Automatic > 0, Manual >= 5
#11 > CarColor

columnNames = ['Cylinders', 'DriveTrain', 'Engine', 'Transmission', 'Color']
specs.columns = columnNames
specs['DriveTrain'] = specs['DriveTrain'].replace('WD,', '', regex=True)
specs['Engine'] =  specs['Engine'].replace('L', '', regex=True)
#specs.to_csv("specsDebug2.csv")



In [617]:
for name in ['Cylinders', 'DriveTrain', 'Transmission']:
    #specs = pd.to_numeric(specs[name], errors='coerce') 
    specs[name] = specs[name].astype(int, error='ignore') # good to check for data consistency
specs['Engine'] = pd.to_numeric(specs['Engine'], errors='coerce')


In [618]:
specs.head(5)

,Cylinders,DriveTrain,Engine,Transmission,Color
0,4,2,1.6,5,Red/Black
1,4,2,2.2,5,Gray/Gray
2,3,2,1.0,0,White/Black
3,4,2,1.6,5,Silver/Black
4,4,2,1.6,5,White/Black


In [489]:
#combination = pd.concat([features, specs], axis=1)
#combination.head(5)

In [634]:
yearMakeModel = df['YearMakeModel'].str.split(' ', n=2, expand=True)
columnNames = ['Year', 'Make', 'Model']
yearMakeModel.columns = columnNames
len(yearMakeModel)

56192

In [635]:
df.drop(['YearMakeModel', 'features', 'specs'], axis=1, inplace=True)

In [636]:
len(df)
df.head(2)

,Price,Mileage,Num_reviews,location,stars
0,"5,599",87K,38,"Salt Lake City, UT",4.5
1,"5,599",127K,28,"Los Angeles, CA",4.0


In [637]:
df['Mileage'] = df['Mileage'].str.replace('K', '').astype(int, error='ignore')
df.rename(columns={'Mileage':'MileageK'}, inplace=True)
df.head(2)

,Price,MileageK,Num_reviews,location,stars
0,"5,599",87,38,"Salt Lake City, UT",4.5
1,"5,599",127,28,"Los Angeles, CA",4.0


In [638]:
specs.head(2)

,Cylinders,DriveTrain,Engine,Transmission,Color
0,4,2,1.6,5,Red/Black
1,4,2,2.2,5,Gray/Gray


In [639]:
copy = pd.concat([yearMakeModel, df, features, specs], axis=1)
copy.to_csv('Copy.csv')
len(copy)

56192

In [651]:

copy['Price'] = copy['Price'].str.replace(',', '').astype(int, error='ignore')
#copy['Engine'] = pd.to_numeric(copy['Engine'], errors='coerce')
#df.to_csv('CleanedData.csv')
copy.head(5)


,Year,Make,Model,Price,MileageK,Num_reviews,location,stars,0,1,...,5,Cylinders,DriveTrain,Engine,Transmission,Color,MakeMachine,ColorMachine,ModelMachine,LocationMachine
0,2009,Hyundai,Accent GS,5599,87,38,"Salt Lake City, UT",4.5,0,6,...,182,4,2,1.6,5,Red/Black,14,56,143,110
1,2006,Saturn,Ion 2,5599,127,28,"Los Angeles, CA",4.0,1,3,...,180,4,2,2.2,5,Gray/Gray,30,31,848,75
2,2008,Smart,Fortwo Pure,5599,86,46,"Los Angeles, CA",4.0,2,78,...,92,3,2,1.0,0,White/Black,32,79,673,75
3,2010,Hyundai,Accent Blue,5899,114,37,"Salisbury, MD",4.5,3,6,...,157,4,2,1.6,5,Silver/Black,14,64,141,109
4,2006,Chevrolet,Aveo LS,5998,73,12,"San Diego, CA",4.5,4,92,...,173,4,2,1.6,5,White/Black,5,79,205,112


In [652]:
makeMachine = pd.Categorical(copy['Make'])
copy['MakeMachine'] = makeMachine.codes

In [653]:
#carMake = dict()
#carMdodel = dict()
#mapCount = 0
#for each in copy['Make'].unique():
#    carMake[str(each)] = mapCount
#    mapCount = mapCount + 1
#carMake

In [654]:
carColor = pd.Categorical(copy['Color'])
copy['ColorMachine'] = carColor.codes

In [655]:
model = pd.Categorical(copy['Model'])
copy['ModelMachine'] = model.codes

In [656]:
location = pd.Categorical(copy['location'])
copy['LocationMachine'] = location.codes

In [657]:
ML_data = copy.drop(['Make', 'Model', 'Color', 'location'], axis=1)
ML_data = ML_data.fillna(0)
ML_data.head(2)
#ML_data.to_csv('MachineData.csv')

,Year,Price,MileageK,Num_reviews,stars,0,1,2,3,4,5,Cylinders,DriveTrain,Engine,Transmission,MakeMachine,ColorMachine,ModelMachine,LocationMachine
0,2009,5599,87,38,4.5,0,6,78,92,179,182,4,2,1.6,5,14,56,143,110
1,2006,5599,127,28,4.0,1,3,0,5,98,180,4,2,2.2,5,30,31,848,75


In [658]:
Y = ML_data['Price']
X= ML_data.drop(['Price'], axis=1)

In [659]:
X.dtypes
Y.dtypes

dtype('int32')

In [ ]:
#ML_data = copy.drop(['Make', 'Model', 'Color', 'location', 'Feature0', 'Feature1'], axis=1)

# 8. Students do: Compete on the CARmax dataset. Ready, set, go!
            
You will be competing on

1. Highest $r^2$ value (on validation data)
2. Fastest runtime in the top third of r-squared values by the end of class
3. Interpretibility - ie which features change prediction in which way.
4. Fastest to complete a model in the top third of r-squared values by the end of class

In [569]:
# X.count() == Y.count()
X.count() 

Year               56192
MileageK           56192
Num_reviews        56192
stars              56192
Feature0           54561
Feature1           52955
Feature2           53987
Feature3           54874
Feature4           55024
Feature5           54702
Cylinders          56192
DriveTrain         56192
Engine             56192
Transmission       56192
MakeMachine        56192
ColorMachine       56192
ModelMachine       56192
LocationMachine    56192
dtype: int64

In [660]:
#df = pd.read_csv('/Users/skazi/Downloads/OnlineNewsPopularity/OnlineNewsPopularity.csv')
#df = df.set_index('url')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.3, random_state=42)
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=.2, random_state=42)

In [566]:
#X_train.describe()

In [564]:
#Y_test.describe()

count    16858.000000
mean     19783.189880
std       8230.139164
min       5599.000000
25%      13998.000000
50%      16998.000000
75%      23599.000000
max      86998.000000
Name: Price, dtype: float64

In [663]:
#My baseline model performance

model = RandomForestRegressor(n_estimators=500, max_depth=None, random_state=310, n_jobs=-1, oob_score=True, verbose=3)
model.fit(X_train, Y_train)



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 500building tree 2 of 500building tree 3 of 500

building tree 4 of 500

building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    3.6s


building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500
building tree 47 of 500
building tree 48 of 500
building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52 of 500
building tree 53 of 500
building tree 54 of 500
building tree 55 of 500
building tree 56 of 500
building tree 57 of 500
building tree 58 of 500
building tree 59 of 500
building tree 60 of 500
building tree 61 of 500
building tree 62 of 500
building tree 63 of 500
building tree 64 of 500
building tree 65 of 500
building tree 66 of 500
building tree 67 of 500
building tree 68 of 500
building tree 69 of 500
building tree 70 of 500
building tree 71 of 500
building tree 72

[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   16.8s



building tree 126 of 500
building tree 127 of 500
building tree 128 of 500
building tree 129 of 500
building tree 130 of 500
building tree 131 of 500
building tree 132 of 500
building tree 133 of 500
building tree 134 of 500
building tree 135 of 500
building tree 136 of 500
building tree 137 of 500
building tree 138 of 500
building tree 139 of 500
building tree 140 of 500
building tree 141 of 500
building tree 142 of 500
building tree 143 of 500
building tree 144 of 500
building tree 145 of 500
building tree 146 of 500
building tree 147 of 500
building tree 148 of 500
building tree 149 of 500
building tree 150 of 500
building tree 151 of 500
building tree 152 of 500
building tree 153 of 500
building tree 154 of 500
building tree 155 of 500
building tree 156 of 500
building tree 157 of 500
building tree 158 of 500
building tree 159 of 500
building tree 160 of 500
building tree 161 of 500
building tree 162 of 500
building tree 163 of 500
building tree 164 of 500
building tree 165 of 500

[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   36.7s


building tree 285 of 500
building tree 286 of 500
building tree 287 of 500
building tree 288 of 500
building tree 289 of 500
building tree 290 of 500
building tree 291 of 500
building tree 292 of 500
building tree 293 of 500
building tree 294 of 500
building tree 295 of 500
building tree 296 of 500
building tree 297 of 500
building tree 298 of 500
building tree 299 of 500
building tree 300 of 500
building tree 301 of 500
building tree 302 of 500
building tree 303 of 500
building tree 304 of 500
building tree 305 of 500
building tree 306 of 500
building tree 307 of 500
building tree 308 of 500
building tree 309 of 500
building tree 310 of 500
building tree 311 of 500
building tree 312 of 500
building tree 313 of 500
building tree 314 of 500
building tree 315 of 500
building tree 316 of 500
building tree 317 of 500
building tree 318 of 500
building tree 319 of 500
building tree 320 of 500
building tree 321 of 500
building tree 322 of 500
building tree 323 of 500
building tree 324 of 500


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.1min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
                      oob_score=True, random_state=310, verbose=3,
                      warm_start=False)

In [680]:
model1 = RandomForestRegressor(n_estimators=400, max_depth=9, random_state=310, n_jobs=-1, oob_score=True, verbose=3)
model1.fit(X_train, Y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 400building tree 2 of 400building tree 3 of 400building tree 4 of 400



building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400
building tree 23 of 400
building tree 24 of 400
building tree 25 of 400
building tree 26 of 400
building tree 27 of 400
building tree 28 of 400

[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.0s


building tree 29 of 400

building tree 30 of 400
building tree 31 of 400
building tree 32 of 400
building tree 33 of 400
building tree 34 of 400
building tree 35 of 400
building tree 36 of 400
building tree 37 of 400
building tree 38 of 400
building tree 39 of 400
building tree 40 of 400
building tree 41 of 400
building tree 42 of 400
building tree 43 of 400building tree 44 of 400

building tree 45 of 400
building tree 46 of 400
building tree 47 of 400
building tree 48 of 400
building tree 49 of 400
building tree 50 of 400
building tree 51 of 400
building tree 52 of 400
building tree 53 of 400
building tree 54 of 400
building tree 55 of 400
building tree 56 of 400
building tree 57 of 400
building tree 58 of 400
building tree 59 of 400
building tree 60 of 400
building tree 61 of 400
building tree 62 of 400
building tree 63 of 400
building tree 64 of 400
building tree 65 of 400
building tree 66 of 400
building tree 67 of 400
building tree 68 of 400
building tree 69 of 400
building tree 7

[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    9.2s


building tree 126 of 400
building tree 127 of 400
building tree 128 of 400
building tree 129 of 400
building tree 130 of 400
building tree 131 of 400
building tree 132 of 400
building tree 133 of 400
building tree 134 of 400
building tree 135 of 400
building tree 136 of 400
building tree 137 of 400
building tree 138 of 400
building tree 139 of 400
building tree 140 of 400
building tree 141 of 400
building tree 142 of 400
building tree 143 of 400
building tree 144 of 400
building tree 145 of 400
building tree 146 of 400
building tree 147 of 400
building tree 148 of 400
building tree 149 of 400
building tree 150 of 400
building tree 151 of 400
building tree 152 of 400
building tree 153 of 400
building tree 154 of 400
building tree 155 of 400
building tree 156 of 400
building tree 157 of 400
building tree 158 of 400
building tree 159 of 400
building tree 160 of 400
building tree 161 of 400
building tree 162 of 400
building tree 163 of 400
building tree 164 of 400
building tree 165 of 400


[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   19.8s


building tree 286 of 400
building tree 287 of 400
building tree 288 of 400
building tree 289 of 400
building tree 290 of 400
building tree 291 of 400
building tree 292 of 400
building tree 293 of 400
building tree 294 of 400
building tree 295 of 400
building tree 296 of 400
building tree 297 of 400
building tree 298 of 400
building tree 299 of 400
building tree 300 of 400
building tree 301 of 400
building tree 302 of 400
building tree 303 of 400
building tree 304 of 400
building tree 305 of 400
building tree 306 of 400
building tree 307 of 400
building tree 308 of 400
building tree 309 of 400
building tree 310 of 400
building tree 311 of 400
building tree 312 of 400
building tree 313 of 400
building tree 314 of 400
building tree 315 of 400
building tree 316 of 400
building tree 317 of 400
building tree 318 of 400
building tree 319 of 400
building tree 320 of 400
building tree 321 of 400
building tree 322 of 400building tree 323 of 400

building tree 324 of 400
building tree 325 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:   27.5s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
                      oob_score=True, random_state=310, verbose=3,
                      warm_start=False)

In [674]:
print(f"Training Data Score: {each} -  {model.score(X_train, Y_train)}")
print(f"Testing Data Score: {each} - {model.score(X_test, Y_test)}")




[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    2.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


Training Data Score:  Pwr Folding Third Row -  0.9923060330360572


[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.5s


Testing Data Score:  Pwr Folding Third Row - 0.9467634358802632


[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.9s finished


In [682]:
print(f"Training Data Score: {each} -  {model1.score(X_train, Y_train)}")
print(f"Testing Data Score: {each} - {model1.score(X_test, Y_test)}")
print(f"Delta Data Score: {each} - {model1.score(X_train, Y_train) - model1.score(X_test, Y_test)}")

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s


Training Data Score:  Pwr Folding Third Row -  0.8967476869127076


[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


Testing Data Score:  Pwr Folding Third Row - 0.8783976359094674


[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.0s


Delta Data Score:  Pwr Folding Third Row - 0.018350051003240164


[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:    0.3s finished
